#### INNER JOIN in SQL

~~~
SELECT p1.country, p1.continent, prime_minister, president
FROM prime_ministers AS p1
INNER JOIN presidents AS p2
ON p1.country = p2.country
~~~


~~~
-- 6. Select fields
SELECT c.code, name, region, e.year, fertility_rate, unemployment_rate
  -- 1. From countries (alias as c)
  FROM countries AS c
  -- 2. Join to populations (as p)
  INNER JOIN populations AS p
    -- 3. Match on country code
    ON c.code = p.country_code
  -- 4. Join to economies (as e)
  INNER JOIN economies AS e
    -- 5. Match on country code and year
    ON c.code = e.code AND p.year = e.year;
~~~

~~~
USING (<field>)
~~~

when the field has the same name in both tables

Self-joins: used to compare values in a field to other values in the same field from within the same table

~~~
SELECT p1.country as country1, p2.country as country2, p1.continent
FROM prime_ministers as p1
INNER JOIN prime_ministers as p2
ON p1.continent = p2.continent AND p1.country <> p2.country
LIMIT 13;
~~~

All pairs of countries form the same continent.

CASE: a way of doing multiple if-then-else statement in SQL

~~~
SELECT name, continent, indep_year
	CASE WHEN indep_year < 1900 THEN 'before 1900'
	WHEN indep_year <= 1930 THEN 'between 1900 and 1930'
	ELSE 'after 1930' END
	AS indep_year_group
FROM states
ORDER BY indep_year_group;
~~~

#### LEFT JOIN

Keeps all records on the left table and adds missing values for those that do not match any keys on the right table.

~~~
SELECT p1.country, prime_minister, president
FROM prime_ministers AS p1
LEFT JOIN presidents AS p2
ON p1.country = p2.country;
~~~

#### RIGHT JOIN

Keeps all records on the right table and adds missing values for those that do not match any keys on the left table.

~~~
SELECT right_table.id AS R_id, left_table.val AS L_val, right_table.id AS L_id
FROM left_table
RIGHT JOIN right_table
ON left_table.id = right_table.id;
~~~

#### FULL JOIN

LEFT JOIN + RIGHT JOIN

~~~
SELECT p1.country AS pm_co, p2.country AS pres_co, prime_minister, president
FROM prime_minister AS p1
FULL JOIN presidents AS p2
~~~

#### CROSS JOIN

Creates all possible combinations between two tables.

~~~
SELECT prime_minister, president
FROM prime_ministers AS p1
CROSS JOIN presidents AS p2
WHERE p1.continent IN ('North America', 'Oceania');
~~~

Cross joins do not use ON or USING!

#### SEMI-JOIN

- Chooses records in the first table where a condition is met in the second table.

~~~
SELECT president, country, continent
FROM presidents
WHERE country IN
	(SELECT name
	 FROM states
	 WHERE indep_year < 1800);
~~~

#### ANTI-JOIN

- Chooses records in the first table where a condition is not met in the second table.

~~~
SELECT president, country, continent
FROM presidents
WHERE continent LIKE '%America'
	AND country NOT IN
		(SELECT name
	 	 FROM states
	 	 WHERE indep_year < 1800);
~~~

### SET THEORY CLAUSES

#### UNION & UNION ALL

- UNION: includes every record in both tables, but DOES NOT double count those that are in both tables.
- UNION ALL: includes every record in both tables and DOES replicate those that are in both tables.
- INTERSECT: results in only those records found in both of the two tables.
- EXCEPT: results in only those records in one table BUT NOT the other.

~~~
SELECT prime_minister AS leader, country
FROM prime_ministers
UNION
SELECT monarch, country
FROM monarchs
ORDER BY country;

SELECT prime_minister AS leader, country
FROM prime_ministers
UNION ALL
SELECT monarch, country
FROM monarchs
ORDER BY country
LIMIT 10;
~~~

#### INTERSECT

~~~
SELECT country
FROM prime_ministers
INTERSECT
SELECT country
FROM presidents;
~~~

~~~
SELECT country, prime_minister AS leader
FROM prime_ministers
INTERSECT
SELECT country, president
FROM presidents
~~~

Returns an empty table, because INTERSECT does not match a specific key, but all selected fields.

#### EXCEPT

Monarchs that aren't prime ministers

~~~
SELECT monarch, country
FROM monarchs
EXCEPT
SELECT prime_minister, country
FROM prime_ministers;
~~~

#### Subquery inside WHERE clause

~~~
SELECT name, fert_rate
FROM states
WHERE continent = 'Asia'
	AND fert_rate <
		(SELECT AVG(fert_rate)
		 FROM states);
~~~

#### Subquery inside SELECT clause

~~~
SELECT DISTINCT continent,
	(SELECT COUNT(*)
	 FROM states
	 WHERE prime_ministers.continent = states.continent) AS countries_num
FROM prime_ministers;
~~~

#### Subquery inside FROM clause

~~~
SELECT DISTINCT monarchs.continent, subquery.max_perc
FROM monarchs,
	(SELECT continent, MAX(women_parli_perc) AS max_perc
	 FROM states
	 GROUP BY continent) as subquery
WHERE monarchs.continent = subquery.continent
ORDER BY continent;
~~~

Advanced subquery

~~~
-- Select fields
SELECT name, continent, inflation_rate
  -- From countries
  FROM countries
	-- Join to economies
	INNER JOIN economies
	-- Match on code
	ON countries.code = economies.code
  -- Where year is 2015
  WHERE year = 2015
    -- And inflation rate in subquery (alias as subquery)
    AND inflation_rate IN (
        SELECT MAX(inflation_rate) AS max_inf
        FROM (
             SELECT name, continent, inflation_rate
             FROM countries
             INNER JOIN economies
             ON countries.code = economies.code
             WHERE year = 2015) AS subquery
        GROUP BY continent);
~~~

~~~
-- Select fields
SELECT c1.name, c1.country_code, c1.city_proper_pop, c1.metroarea_pop,  
      -- Calculate city_perc
      c1.city_proper_pop / c1.metroarea_pop * 100 AS city_perc
  -- From appropriate table
  FROM cities as c1
  -- Where 
  WHERE c1.name IN
    -- Subquery
    (SELECT capital
     FROM countries as c2
     WHERE (c2.continent = 'Europe'
        OR c2.continent LIKE '%America'))
       AND c1.metroarea_pop IS NOT NULL
-- Order appropriately
ORDER BY city_perc DESC
-- Limit amount
LIMIT 10;
~~~